In [1]:
import tensorflow as tf
import os
os.environ["TF_USE_LEGACY_KERAS"]="1"
import keras
import tensorflow_model_optimization as tfmot
from sklearn.metrics.cluster import entropy
from tf_keras import layers,losses,optimizers,Sequential
from tf_keras.models import Model
from tf_keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D,Input
from tf_keras.applications import VGG16
from tf_keras import datasets,models
import importlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from importing_dataset import load_dataset
import Entropy
importlib.reload(Entropy)
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

INFO:absl:Load dataset info from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0
INFO:absl:Creating a tf.data.Dataset reading 2 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Constructing tf.data.Dataset horses_or_humans for split ('train[:70%]', 'train[70%:]'), from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0


In [4]:
from reformatting import reformat_image
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_batches = train_examples.cache().shuffle(num_examples//4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [7]:

from tf_keras import backend as K
# Assurez-vous que l'entrée est un KerasTensor
input_layer = K.placeholder(shape=(None, 224, 224, 3), dtype='float32')


# Bloc 1
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_layer)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Bloc 2
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Bloc 3
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Bloc 4
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Bloc 5
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# Ajouter des couches supplémentaires
x = layers.GlobalMaxPooling2D(name="global_max_pool")(x)
x = layers.Dropout(0.3, name="dropout")(x)
outputs = layers.Dense(2, activation="softmax", name="predictions")(x)

# Construire le modèle
model = Model(input_layer, outputs, name="custom_vgg16.keras")
model.summary()

# Charger les poids de VGG16
vgg16_base = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

from test_surgeon import Surgeon
# Appliquer les poids pour chaque couche manuellement
for layer, pretrained_layer in zip(model.layers, vgg16_base.layers):
    if isinstance(layer, layers.Conv2D):
        layer.set_weights(pretrained_layer.get_weights())

input_layer = model.input  # L'entrée du modèle
layer_0 = model.layers[1]  # Index 1 car la première couche est 'input_40'
surgeon = Surgeon(model)

# Si la couche 0 est une Conv2D, supprimer les canaux
if isinstance(layer_0, layers.Conv2D):
    # Surgeon pour supprimer les canaux
    surgeon.add_job('delete_channels', layer_0, channels=[0, 1, 2])
    pruned_model = surgeon.operate()

    # Redéfinir les entrées et sorties du modèle après modification
    pruned_model = Model(inputs=input_layer, outputs=pruned_model.output)

    # Redéfinir les entrées du modèle (pour qu'elles soient un KerasTensor valide)
    pruned_model.build(input_shape=(None, 224, 224, 3))

    # Recompiler le modèle pour qu'il soit prêt à l'entraînement ou à la prédiction
    pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


else:
    print("La couche spécifiée n'est pas une couche Conv2D.")

pruned_model.summary()


Model: "custom_vgg16.keras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 61)      1708      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      35200     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [48]:
import Entropy_Keras_Surgeon
importlib.reload(Entropy_Keras_Surgeon)
from Entropy_Keras_Surgeon import EntropyPruningSurgeon
testt_model = models.load_model("model_experiments/custom_vgg16.keras")
pruning = EntropyPruningSurgeon(model=testt_model, threshold=0.026)
pruningmodel = pruning.run()
pruningmodel.summary()


[0.17269282 0.08593657 0.06639509 0.15977614 0.12494584 0.11560801
 0.08723375 0.1031163  0.1304353  0.08468138 0.1758175  0.09220949
 0.14930637 0.08204485 0.13381846 0.10183211 0.08377397 0.08114591
 0.10910635 0.09520047 0.11484847 0.13169202 0.08065336 0.13898827
 0.04546823 0.09654906 0.15572321 0.08630413 0.13806152 0.08832835
 0.10611238 0.08109367 0.05137431 0.08372652 0.15003803 0.14347923
 0.07392103 0.15742144 0.0875589  0.07336573 0.06497415 0.15371111
 0.09803167 0.13182561 0.0832122  0.19951327 0.09752656 0.1636866
 0.11984214 0.13067804 0.07287492 0.18149455 0.06964114 0.11167739
 0.12075619 0.07491273 0.08534266 0.1264338  0.15043746 0.11684294
 0.13868542 0.07888904 0.17409943 0.13589956]
Pruned indices for layer 'block1_conv1': []
[0.19925979 0.13174167 0.13938427 0.14332995 0.14302185 0.19506392
 0.1656115  0.10986789 0.14186813 0.09592294 0.19612369 0.12438349
 0.18760093 0.2106316  0.07597923 0.13958845 0.14041013 0.18103084
 0.20223175 0.216287   0.19713873 0.1304

In [44]:
# Test the model
pruningmodel.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])


In [45]:
pruningmodel.evaluate(validation_batches)

10/10 [==============================] - 41s 4s/step - loss: 2.6552 - accuracy: 0.8669


[2.6551754474639893, 0.8668830990791321]

In [53]:
import tempfile
def prune_model(model, epochs,threshold, train_data=train_batches, val_data=validation_batches,  feature_extractor=model):

  # Create a tensorboard logfile
  logdir = tempfile.mkdtemp()
  # The end_step is the total number of iterations required for the training data which is basically the entire epochs over the length of the training data
  end_step = int(len(train_data) * epochs * 0.5)

  learning_rate_fn = optimizers.schedules.PolynomialDecay(
    0.001,
    1000,
    0.0001,
    power=0.5)

  # Model for pruning
  entropy_pruning_instance = EntropyPruningSurgeon(model=model, threshold=threshold)
  model_for_pruning = entropy_pruning_instance.run()

  # Recompile
  model_for_pruning.compile(optimizer= optimizers.Adam(),
                            loss= losses.SparseCategoricalCrossentropy(),
                            metrics=["accuracy"])

  # Fit the model
  model_for_pruning.fit(train_data,
                      validation_data=val_data,
                      batch_size=BATCH_SIZE,
                      epochs=epochs)

  # Save the model
  #model_for_pruning.save(f"mnist_model_sparsity_{final_sparsity}")

  # Evaluate the model
  score = model_for_pruning.evaluate(val_data, verbose=0)
  metric_dict = {
      "sparsity" : final_sparsity,
      "val_loss" : np.round(score[0], 4),
      "val_accuracy" : np.round(score[1] * 100, 4)
  }
  return logdir, metric_dict, model_for_pruning

In [55]:
# Choose the best entropy level
k_entropies = np.arange(0.01, 0.031, 0.001)

#k_sparsities = [0.50]
metric_list = []

for k in k_entropies:
  # Load in the best saved model
  model_1 = models.load_model("model_experiments/model_horses_or_humans.keras")
  logdir, metrics, pruned_model = prune_model(model=model_1,
            threshold=k,
            epochs=20)
  val_loss, val_accuracy  = metrics["val_loss"], metrics["val_accuracy"]
  metric_list.append(metrics)
  print(f"Sparsity : {k} \tValidation Loss: {val_loss}, \tValidation Accuracy: {val_accuracy}")

# Create a dataframe of the values obtained
df = pd.DataFrame(metric_list)

Epoch 1/20
 3/23 [==>...........................] - ETA: 1:46 - loss: 0.0000e+00 - accuracy: 1.0000

KeyboardInterrupt: 

In [192]:
from tf_keras import optimizers
import tempfile
import tensorflow_model_optimization as tfmot
# Finish pruning after 2 epochs
epochs = 2
BATCH_SIZE = 32

def prune_model(model, initial_sparsity, final_sparsity, train_data=train_batches, val_data=validation_batches, epochs=epochs, feature_extractor=model ):

  # Create a tensorboard logfile
  logdir = tempfile.mkdtemp()
  # The end_step is the total number of iterations required for the training data which is basically the entire epochs over the length of the training data
  end_step = int(len(train_data) * epochs * 0.5)
  # Import the low-magnitude-pruning function
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Set the prunung params
  pruning_params = {

      "pruning_schedule" : tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
                                                                final_sparsity=final_sparsity,
                                                                begin_step=0,
                                                                end_step=end_step)

                  }


  learning_rate_fn = optimizers.schedules.PolynomialDecay(
    0.001,
    1000,
    0.0001,
    power=0.5)

  # Model for pruning
  #feature_extractor = prune_low_magnitude(feature_extractor, **pruning_params)
  model_for_pruning = prune_low_magnitude(model, **pruning_params)

  # Recompile
  model_for_pruning.compile(optimizer= optimizers.Adam(),
                            loss= losses.SparseCategoricalCrossentropy(),
                            metrics=["accuracy"])
  #create callbacks
  callbacks = [tfmot.sparsity.keras.UpdatePruningStep(),
              tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
               #create_model_checkpoint(model_name=model.name),
              #tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                  #patience=3,
                                                  #verbose=1),
               #tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                        #patience=4,
                                                        #restore_best_weights=True)
                                                        ]

  # Fit the model
  model_for_pruning.fit(train_data,
                      validation_data=val_data,
                      batch_size=BATCH_SIZE,
                      epochs=epochs,
                      callbacks=callbacks)

  # Save the model
  #model_for_pruning.save(f"mnist_model_sparsity_{final_sparsity}")

  # Evaluate the model
  score = model_for_pruning.evaluate(val_data, verbose=0)
  metric_dict = {
      "sparsity" : final_sparsity,
      "val_loss" : np.round(score[0], 4),
      "val_accuracy" : np.round(score[1] * 100, 4)
  }
  return logdir, metric_dict, model_for_pruning

In [203]:
# Choose the best sparsity level
final_sparsity = 0.80

logdir, metrics, pruned_model = prune_model(model=pruned_model,
          initial_sparsity=0,
          final_sparsity=final_sparsity,
          epochs=3)

Epoch 1/3
23/23 [==============================] - 159s 7s/step - loss: 0.6922 - accuracy: 0.5257 - val_loss: 0.6947 - val_accuracy: 0.4838
Epoch 2/3
23/23 [==============================] - 174s 8s/step - loss: 0.6920 - accuracy: 0.5257 - val_loss: 0.6949 - val_accuracy: 0.4838
Epoch 3/3
23/23 [==============================] - 145s 6s/step - loss: 0.6920 - accuracy: 0.5257 - val_loss: 0.6949 - val_accuracy: 0.4838


In [196]:
model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

C:\Users\neyen\AppData\Local\Temp\ipykernel_71452\369930696.py:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\neyen\AppData\Local\Temp\tmpjh1lij65.h5
